In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

In [41]:
id=2384540
pd.read_csv(f'/glade/work/guoqiang/CTSM_CAMELS/data_paramcailb/ParamCalib_basinparam_{id}.csv')

,Unnamed: 0,Type,Process,PPE_DailyQ_Rank,Parameter,Default_glob,Lower_glob,Upper_glob,Lower_raw,Upper_raw,LowUp_vsPPE,Source,Binding,Method,Default,Lower,Upper
0,0,Hydrology,Infiltration\r\nSoil storage,1,fff,5.000000e-01,2.000000e-02,5.000000e+00,2.000000e-02,5.000000e+00,Same,Param,NaN,Multiplicative,5.000000e-01,2.000000e-02,5.000000e+00
1,1,Hydrology,Infiltration,0,FMAX,3.816694e-01,1.908347e-01,5.725041e-01,1.000000e-01,9.000000e-01,NotInPPE,Surfdata,NaN,Multiplicative,3.665199e-01,1.832600e-01,5.497799e-01
2,2,Stomatal resistance and photosynthesis,ET (transpiration),2,medlynslope,4.840380e+00,2.420190e+00,7.260570e+00,3.235949e+00,6.934051e+00,BiggerRange,Param,NaN,Multiplicative,4.840380e+00,2.420190e+00,7.260570e+00
3,3,Hydrology,Soil storage capacity,11,sucsat_sf,1.000000e+00,1.000000e-01,1.000000e+01,1.000000e-01,1.000000e+01,Same,Param,NaN,Multiplicative,1.000000e+00,1.000000e-01,1.000000e+01
4,4,Hydrology,Soil water drainage,10,hksat_sf,1.000000e+00,1.000000e-01,1.000000e+01,1.000000e-01,1.000000e+01,Same,Param,NaN,Multiplicative,1.000000e+00,1.000000e-01,1.000000e+01
5,5,Hydrology,Soil storage capacity,9,watsat_sf,1.000000e+00,2.500000e-01,2.000000e+00,8.000000e-01,1.200000e+00,BiggerRange,Param,NaN,Multiplicative,1.000000e+00,2.500000e-01,2.000000e+00
6,6,Hydrology,Infiltration,7,d_max,1.500000e+01,5.000000e+00,1.000000e+02,1.000000e+01,6.000000e+01,BiggerRange_FromYifan,Param,NaN,Multiplicative,1.500000e+01,5.000000e+00,1.000000e+02
7,7,Hydrology,Canopy Interception,45,interception_fraction,1.000000e+00,1.000000e-01,1.000000e+00,1.000000e-01,1.000000e+00,Low0.5to0.1,Namelist,NaN,Multiplicative,1.000000e+00,1.000000e-01,1.000000e+00
8,8,Stomatal resistance and photosynthesis,Plant hydraulics,15,kmax,1.910000e-08,2.270000e-09,3.790000e-08,2.270000e-09,3.790000e-08,Same,Param,NaN,Multiplicative,1.914249e-08,2.275050e-09,3.798431e-08
9,9,Stomatal resistance and photosynthesis,Photosynthesis,16,jmaxb0,3.110000e-02,1.000000e-02,5.000000e-02,1.000000e-02,5.000000e-02,Same,Param,NaN,Multiplicative,3.110000e-02,1.000000e-02,5.000000e-02
